In [2]:
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.utils.np_utils import to_categorical
import _pickle as cPickle 
import pandas as pd

from keras.models import load_model
import matplotlib.pyplot as plt

In [3]:
body = pd.read_csv("data/train_bodies.csv")
stances = pd.read_csv("data/train_stances.csv")
data = pd.merge(body, stances, how='right', on='Body ID')
targets = ['agree', 'disagree', 'discuss', 'unrelated']
targets_dict = dict(zip(targets, range(len(targets))))
# data['target'] = data['Stance']
data['target'] = data['Stance'].replace(targets,targets_dict.values())
data_y = data['target'].values

In [4]:
data.head()

,Body ID,articleBody,Headline,Stance,target
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated,3
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated,3
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated,3
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated,3
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated,3


In [5]:
train = data
train.head()

,Body ID,articleBody,Headline,Stance,target
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated,3
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated,3
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated,3
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated,3
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated,3


In [8]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors
# glove2word2vec(glove_input_file="glove.6B.50d.txt", word2vec_output_file="gensim_glove_vectors.txt")
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
def process(x):
    word_tokens = word_tokenize(x)
    embedding = [glove_model.index2word.index(word.lower()) for word in word_tokens if word not in stop_words and word.lower() in glove_model.index2word] 
    return embedding

In [9]:
train['w2v'] = (train['Headline'] + '. ' + train['articleBody']).map(process)
train.head()

,Body ID,articleBody,Headline,Stance,target,w2v
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated,3,"[2481, 635, 1, 668, 5304, 6990, 27543, 136, 25..."
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated,3,"[3439, 5929, 4548, 11539, 300, 3071, 11539, 24..."
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated,3,"[7298, 34765, 848, 1894, 1266, 5402, 57, 2, 7,..."
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated,3,"[2923, 45, 2481, 635, 136, 2520, 6270, 2, 7, 3..."
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated,3,"[1752, 273631, 21900, 8867, 904, 2155, 1921, 9..."


In [10]:
X = train['w2v']
y = train['target']

In [11]:
from keras.preprocessing.sequence import pad_sequences
X_pad = pad_sequences(X, padding='post', maxlen=300)
y_pad = keras.utils.to_categorical(y)

In [12]:
vocab_size = 400000
embedding_dim = 50

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_pad, test_size=0.2, random_state=42)

In [19]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[glove_model.wv.vectors], 
                           input_length=300, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, validation_split=0.33, batch_size=16, epochs=100)

/home/gjx/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 300, 50)           20000000  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 296, 128)          32128     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_12 (Dense)             (None, 4)                 44        
Total params: 20,033,462
Trainable params: 33,462
Non-trainable params: 20,000,000
_________________________________________________________________
Train on 26784 samples, validate on 13193 samples
Epoch 1/100
26784/26784 [==============================] - 40s 1ms/step - l

26784/26784 [==============================] - 41s 2ms/step - loss: 0.2854 - acc: 0.8986 - val_loss: 0.4183 - val_acc: 0.8602
Epoch 53/100
26784/26784 [==============================] - 50s 2ms/step - loss: 0.2877 - acc: 0.8952 - val_loss: 0.4097 - val_acc: 0.8558
Epoch 54/100
26784/26784 [==============================] - 45s 2ms/step - loss: 0.2844 - acc: 0.8959 - val_loss: 0.4644 - val_acc: 0.8560
Epoch 55/100
26784/26784 [==============================] - 60s 2ms/step - loss: 0.2820 - acc: 0.8990 - val_loss: 0.4484 - val_acc: 0.8560
Epoch 56/100
26784/26784 [==============================] - 56s 2ms/step - loss: 0.2823 - acc: 0.8983 - val_loss: 0.4528 - val_acc: 0.8573
Epoch 57/100
26784/26784 [==============================] - 42s 2ms/step - loss: 0.2795 - acc: 0.8995 - val_loss: 0.4194 - val_acc: 0.8565
Epoch 58/100
26784/26784 [==============================] - 51s 2ms/step - loss: 0.2818 - acc: 0.8970 - val_loss: 0.4795 - val_acc: 0.8565
Epoch 59/100
26784/26784 [==============

In [20]:
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm


def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))


def report_score(actual,predicted):
    score,cm = score_submission(actual,predicted)
    best_score, _ = score_submission(actual,actual)

    print_confusion_matrix(cm)
    print("Score: " +str(score) + " out of " + str(best_score) + "\t("+str(score*100/best_score) + "%)")
    return score*100/best_score


In [28]:
actual = np.argmax(y_train, axis=1)
predicted = np.argmax(model.predict(X_train), axis=1)
report_score([LABELS[e] for e in actual],[LABELS[e] for e in predicted])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |   2025    |    38     |    91     |    745    |
-------------------------------------------------------------
| disagree  |    365    |    135    |    19     |    179    |
-------------------------------------------------------------
|  discuss  |    377    |     4     |   5066    |   1646    |
-------------------------------------------------------------
| unrelated |    128    |     7     |    292    |   28860   |
-------------------------------------------------------------
Score: 14664.5 out of 18011.75	(81.4162976945605%)


81.4162976945605

In [29]:
actual = np.argmax(y_test, axis=1)
predicted = np.argmax(model.predict(X_test), axis=1)
report_score([LABELS[e] for e in actual],[LABELS[e] for e in predicted])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    432    |    22     |    49     |    276    |
-------------------------------------------------------------
| disagree  |    67     |    13     |     8     |    54     |
-------------------------------------------------------------
|  discuss  |    121    |     2     |   1163    |    530    |
-------------------------------------------------------------
| unrelated |    44     |     6     |    150    |   7058    |
-------------------------------------------------------------
Score: 3439.75 out of 4551.5	(75.5739865978249%)


75.5739865978249